In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Kidney_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Kidney_Disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_data: Path

In [6]:
from src.Kidney_Disease_Classifier.constants import *
from src.Kidney_Disease_Classifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_data=config.unzip_data
        )

        return data_ingestion_config


In [9]:
import os
import gdown
import zipfile
from src.Kidney_Disease_Classifier import logger
from src.Kidney_Disease_Classifier.utils.common import get_size


In [10]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    
    def donwload_data(self):
        """Fetch data from database"""

        try:
            dataset_url = self.config.source_url
            zipfile_dir = self.config.local_data_file

            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading data from {dataset_url}")

            file_url = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="

            gdown.download(prefix+file_url,zipfile_dir)

            logger.info(f"Downloaded data from {dataset_url}")

        except Exception as e:
            raise e
        
    def unzip_file(self):
        """unzip data after downloading """

        unzip_data_filepath = self.config.unzip_data
        os.makedirs(unzip_data_filepath,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_data_filepath)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.donwload_data()
    data_ingestion.unzip_file()
except Exception as e:
    raise e

[2023-11-10 22:05:38,078 - INFO - common - Reading data is completed form config\config.yaml]
[2023-11-10 22:05:38,087 - INFO - common - Reading data is completed form params.yaml]
[2023-11-10 22:05:38,089 - INFO - common - Directory is created for artifacts]
[2023-11-10 22:05:38,091 - INFO - common - Directory is created for artifacts/data_ingestion]
[2023-11-10 22:05:38,093 - INFO - 514323005 - Downloading data from https://drive.google.com/file/d/1n1WYibOlWcTc-coQ8mPcHipvxWMfir0j/view?usp=sharing]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1n1WYibOlWcTc-coQ8mPcHipvxWMfir0j
From (redirected): https://drive.google.com/uc?/export=download&id=1n1WYibOlWcTc-coQ8mPcHipvxWMfir0j&confirm=t&uuid=2f79fee8-8195-49f1-bc2f-e7d1455e08e4
To: d:\Projects\Kidney_Disease_Classification\artifacts\data_ingestion\data.zip
100%|██████████| 57.7M/57.7M [00:05<00:00, 9.67MB/s]

[2023-11-10 22:05:45,460 - INFO - 514323005 - Downloaded data from https://drive.google.com/file/d/1n1WYibOlWcTc-coQ8mPcHipvxWMfir0j/view?usp=sharing]
